# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
import azureml.core
from azureml.exceptions import ComputeTargetException
from azureml.core.compute import ComputeTarget, AmlCompute
import azureml.core
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
import joblib
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig
import pandas as pd

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.
The dataset we will be using in this project is called the Titanic dataset and is publicly available from a Kaggle competition.

Using this dataset we aim to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (ie name, age, gender, socio-economic class, etc).

### Task
The task we are concerned with is to use machine learning to create a model that predicts which passengers survived the Titanic shipwreck. We will be using [Survived] column as the target and since this is a boolean variable, the task at hand is Binary Classification.

The 12 input features and the target feature are as follows:
- PassengerId
- Survived
- Pclass
- Name
- Sex
- Age
- SibSp
- Parch
- Ticket
- Fare
- Cabin
- Embarked



TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'udacity-capstone-project'

experiment=Experiment(ws, experiment_name)

In [8]:
 from azureml.core import Dataset
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Titanic Dataset"
description_text = "Titanic DataSet for Capstone Project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/ratheswaran/directory_name/main/train.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,23.799293,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,17.596074,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,6.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,24.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [9]:
dataset.take(5).to_pandas_dataframe()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,X,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,X,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,X,S


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

As mentioned above in the dataset section we are dealing with a binary classification. Therefore the argument task is set to classification and the since we are predicting `Survived` we need to set `label_column_name="Survived"`

To help manage child runs and when they can be performed, we recommend you create a dedicated cluster per experiment, and match the number of max_concurrent_iterations of your experiment to the number of nodes in the cluster. This way, you use all the nodes of the cluster at the same time with the number of concurrent child runs/iterations you want.

Configure max_concurrent_iterations in your AutoMLConfig object. If it is not configured, then by default only one concurrent child run/iteration is allowed per experiment.

Besides other arguments that are self-explanatory, to automate Feature engineering AzureML enables this through featurization that needs to be set to True. This way features that best characterize the patterns in the data are selected to create predictive models.

In [10]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "aml-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [13]:
project_folder = './automl-project'

# AutoMl settings
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric": "accuracy",
    "enable_early_stopping": True,
    "verbosity": logging.INFO
}

# AutoMl config
automl_config = AutoMLConfig(compute_target=amlcompute_cluster_name,
                             task="classification",
                             training_data=dataset,
                             label_column_name="Survived",
                             n_cross_validations=5,
                             featurization="auto",
                             path=project_folder,
                             debug_log = "automl_errors.log",
                             **automl_settings                             
                            )

In [14]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-capstone-project,AutoML_72030f7f-807e-4c63-a3aa-7a3f904284d3,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


{'runId': 'AutoML_72030f7f-807e-4c63-a3aa-7a3f904284d3',
 'target': 'aml-compute',
 'status': 'Completed',
 'startTimeUtc': '2023-03-18T06:48:33.201256Z',
 'endTimeUtc': '2023-03-18T07:13:25.370527Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'aml-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-capstone-project","subscription_id":"976ee174-3882-4721-b90a-b5fef6b72f24","resource_group":"aml-quickstarts-228138","workspace_name":"quick-starts-ws-228138","region":"southcentralus","compute_target":"aml-c

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [15]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-capstone-project,AutoML_72030f7f-807e-4c63-a3aa-7a3f904284d3,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feat

{'runId': 'AutoML_72030f7f-807e-4c63-a3aa-7a3f904284d3',
 'target': 'aml-compute',
 'status': 'Completed',
 'startTimeUtc': '2023-03-18T06:48:33.201256Z',
 'endTimeUtc': '2023-03-18T07:13:25.370527Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'aml-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-capstone-project","subscription_id":"976ee174-3882-4721-b90a-b5fef6b72f24","resource_group":"aml-quickstarts-228138","workspace_name":"quick-starts-ws-228138","region":"southcentralus","compute_target":"aml-c

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [16]:
from pprint import pprint
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

In [17]:
best_run, best_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()

In [18]:
best_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('32', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='l1')), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.8, eta=0.05, gamma=0.01, grow_policy='lossguide', max_bin=255, max_depth=6, max_leaves=15, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0.9375, reg_lambda=1.6666666666666667, subsample=0.6, tree_method='hist'))], verbose=False

In [19]:
print_model(best_model)

datatransformer
{'enable_dnn': False,
 'enable_feature_sweeping': True,
 'feature_sweeping_config': {},
 'feature_sweeping_timeout': 86400,
 'featurization_config': None,
 'force_text_dnn': False,
 'is_cross_validation': True,
 'is_onnx_compatible': False,
 'observer': None,
 'task': 'classification',
 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook228138/code/Users/odl_user_228138'}

prefittedsoftvotingclassifier
{'estimators': ['32', '11', '36', '18', '40', '5', '6', '41', '3', '20'],
 'weights': [0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091,
             0.18181818181818182,
             0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091]}

32 - sparsenormalizer
{'copy': True, 'norm': 'l1'}

32 - xgboostclassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,

In [20]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-capstone-project,AutoML_72030f7f-807e-4c63-a3aa-7a3f904284d3_49,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [21]:
best_run_metrics

{'recall_score_micro': 0.8451007469713139,
 'f1_score_weighted': 0.8426729434841856,
 'average_precision_score_weighted': 0.8684687814872198,
 'accuracy': 0.8451007469713139,
 'balanced_accuracy': 0.8240380539982167,
 'precision_score_macro': 0.8427332783203527,
 'norm_macro_recall': 0.6480761079964338,
 'average_precision_score_micro': 0.8644184305276734,
 'AUC_micro': 0.884861684070113,
 'recall_score_macro': 0.8240380539982167,
 'matthews_correlation': 0.666360515853188,
 'AUC_macro': 0.869703398445672,
 'f1_score_macro': 0.82972670200037,
 'precision_score_micro': 0.8451007469713139,
 'log_loss': 0.4560810441108674,
 'AUC_weighted': 0.869703398445672,
 'recall_score_weighted': 0.8451007469713139,
 'f1_score_micro': 0.8451007469713137,
 'precision_score_weighted': 0.846952431153049,
 'average_precision_score_macro': 0.8624782091781766,
 'weighted_accuracy': 0.8631771408927909,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_72030f7f-807e-4c63-a3aa-7a3f904284d3_49/co

In [22]:
best_run.get_details()

{'runId': 'AutoML_72030f7f-807e-4c63-a3aa-7a3f904284d3_49',
 'target': 'aml-compute',
 'status': 'Completed',
 'startTimeUtc': '2023-03-18T07:12:00.099001Z',
 'endTimeUtc': '2023-03-18T07:13:17.776239Z',
 'services': {},
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'udacity-capstone-project\',\'compute_target\':\'aml-compute\',\'subscription_id\':\'976ee174-3882-4721-b90a-b5fef6b72f24\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_72030f7f-807e-4c63-a3aa-7a3f904284d3_49","experiment_name":"udacity-capstone-project","workspace_name":"quick-starts-ws-228138","subscr

In [23]:
best_run.get_properties()

{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'udacity-capstone-project\',\'compute_target\':\'aml-compute\',\'subscription_id\':\'976ee174-3882-4721-b90a-b5fef6b72f24\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_72030f7f-807e-4c63-a3aa-7a3f904284d3_49","experiment_name":"udacity-capstone-project","workspace_name":"quick-starts-ws-228138","subscription_id":"976ee174-3882-4721-b90a-b5fef6b72f24","resource_group_name":"aml-quickstarts-228138"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '49',
 '_aml_system_scenario_identification': 'Remote.Child',
 '_azu

In [24]:
#TODO: Save the best model
joblib.dump(best_model, filename="./best_automl_model.joblib")

['./best_automl_model.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [25]:
from azureml.core.model import Model, InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.automl.core.shared import constants
from azureml.core.environment import Environment

In [26]:
model = Model.register(workspace=ws,
                       model_name="titanic_pred_model", 
                       model_path="./best_automl_model.joblib",
                       description="Best AutoML model"
                      )

Registering model titanic_pred_model


TODO: In the cell below, send a request to the web service you deployed to test it.

In [28]:
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
env = Environment.from_conda_specification(name="myenv", file_path="./myenv.yml")

#env = best_run.get_environment()

inf_config = InferenceConfig(environment=env,
                             entry_script='./score.py')

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                       memory_gb=1,
                                                       auth_enabled=False,
                                                       enable_app_insights=True)

service = Model.deploy(workspace=ws, 
                       name="automl-service",
                       models=[model],
                       inference_config=inf_config,
                       deployment_config=deployment_config)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-03-18 07:33:51+00:00 Creating Container Registry if not exists..
2023-03-18 07:43:52+00:00 Registering the environment..
2023-03-18 07:43:55+00:00 Building image..
2023-03-18 08:04:13+00:00 Generating deployment configuration.
2023-03-18 08:04:13+00:00 Submitting deployment to compute.
2023-03-18 08:04:18+00:00 Checking the status of deployment automl-service..
2023-03-18 08:06:32+00:00 Checking the status of inference endpoint automl-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, print the logs of the web service and delete the service

In [29]:
print(f"Service State: {service.state}\n")
print(f"Scoring URI: {service.scoring_uri}\n")
print(f"Swagger URI: {service.swagger_uri}\n")

Service State: Healthy

Scoring URI: http://b4e18649-1890-48c3-88c3-491f37bda39b.southcentralus.azurecontainer.io/score

Swagger URI: http://b4e18649-1890-48c3-88c3-491f37bda39b.southcentralus.azurecontainer.io/swagger.json



In [35]:
import json
# URL for the web service
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
#key = ""

# 3 sets of data to score, so we get two results back
test_path = "https://raw.githubusercontent.com/ratheswaran/directory_name/main/test.csv"
test_dataset = Dataset.Tabular.from_delimited_files(test_path)   
test_df = test_dataset.to_pandas_dataframe().sample(n=5)


# Convert to JSON string
input_data = json.dumps({"data": test_df.to_dict(orient='records')})
with open("input_data.json", 'w') as _f:
    _f.write(input_data)

print(input_data)

{"data": [{"PassengerId": 1034, "Pclass": 1, "Name": "Ryerson, Mr. Arthur Larned", "Sex": "male", "Age": 61.0, "SibSp": 1, "Parch": 3, "Ticket": "PC 17608", "Fare": 262.375, "Cabin": "B57 B59 B63 B66", "Embarked": "C"}, {"PassengerId": 1254, "Pclass": 2, "Name": "Ware, Mrs. John James (Florence Louise Long)", "Sex": "female", "Age": 31.0, "SibSp": 0, "Parch": 0, "Ticket": "CA 31352", "Fare": 21.0, "Cabin": "X", "Embarked": "S"}, {"PassengerId": 1129, "Pclass": 3, "Name": "Baccos, Mr. Raffull", "Sex": "male", "Age": 20.0, "SibSp": 0, "Parch": 0, "Ticket": "2679", "Fare": 7.225, "Cabin": "X", "Embarked": "C"}, {"PassengerId": 915, "Pclass": 1, "Name": "Williams, Mr. Richard Norris II", "Sex": "male", "Age": 21.0, "SibSp": 0, "Parch": 1, "Ticket": "PC 17597", "Fare": 61.3792, "Cabin": "X", "Embarked": "C"}, {"PassengerId": 904, "Pclass": 1, "Name": "Snyder, Mrs. John Pillsbury (Nelle Stevenson)", "Sex": "female", "Age": 23.0, "SibSp": 1, "Parch": 0, "Ticket": "21228", "Fare": 82.2667, "Ca

In [36]:
import requests
# Set the content type
headers = {"Content-Type": "application/json"}

# If authentication is enabled, set the authorization header
#headers["Authorization"] = f"Bearer {key}"

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

[0, 1, 0, 0, 1]


In [34]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-03-18T08:06:21,304481337+00:00 - iot-server/run 
2023-03-18T08:06:21,304487143+00:00 - rsyslog/run 
2023-03-18T08:06:21,310906968+00:00 - gunicorn/run 
2023-03-18T08:06:21,324635130+00:00 | gunicorn/run | 
2023-03-18T08:06:21,332142595+00:00 | gunicorn/run | ###############################################
2023-03-18T08:06:21,334591789+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-03-18T08:06:21,337671102

In [ ]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
